In [5]:
# the original code is in the SentenceClassification.ipytn file. I am eliminating many variables in this file.


import glob
import pandas as pd
import json
import csv
from bs4 import BeautifulSoup, NavigableString
import nltk.data

path = r'Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

path = r'Train'  # use your path
allFiles = glob.glob(path + "/*.csv")



allFiles = allFiles[:65]


# all the followint positives are to be counted if the sentences are in that particular class. 
# A sen_neg to be counted in any other circumstances
sen0_pos = 0 # action_first_party no. of sentence positives
sen0_neg = 0 # action_first_party no. of sentence negatives
sen_actioFP = []
sen_actioFP_class = []

sen1_pos = 0 # Personal Information Type no. of sentence positives
sen1_neg = 0 # action_first_party no. of sentence negatives
sen_personalIT = []
sen_personalIT_class = []

sen2_pos = 0 # purpose no. of sentence negatives
sen2_neg = 0 # purpose no. of sentence negatives
sen_purpose = []
sen_purpose_class = []

sen3_pos = 0

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head tag
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
                start_index.append(count)
    doc = '\n'.join(text)
    return doc



for file in allFiles:
    
    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    number_of_segments = len(df) + 1

    file = file.split('Train/', 1)[1]

    ### beautifulsuop on html doc
    file_html = 'Plain_Html/'+file
    file_html = file_html.split('csv', 1)[0]
    file_html = file_html+'html'
    ls = []
    indeces = []
    data = html_to_text(file_html)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
        count = count+len(l)

    for i in range(number_of_segments-1):
        choice = 0 #whether the sentence is from one of the following categories
        if df[5][i] == "First Party Collection/Use":
            parse_json = json.loads(str(df[6][i]))
            # parsing and fetching the text related to "Action First Party"            
            if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
                choice = 1
                str_index = parse_json["Action First-Party"]["startIndexInSegment"]
                end_index = parse_json["Action First-Party"]["endIndexInSegment"]
                for index in range(len(ls)):
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("pos")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")
                        sen0_pos += 1
                        sen1_neg += 1
                        sen2_neg += 1
                    else: 
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_neg += 1
            # parsing and fetching the text related to "Personal Information Type"                        
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                for index in range(len(ls)):
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("pos")                        
                        sen_purpose_class.append("neg")  
                        sen0_neg += 1
                        sen1_pos += 1
                        sen2_neg += 1
                    else: 
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")  
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_neg += 1
            # parsing and fetching the text related to "Purpose"                
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                choice = 1            
                str_index = parse_json["Purpose"]["startIndexInSegment"]
                end_index = parse_json["Purpose"]["endIndexInSegment"]
                for index in range(len(ls)):
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("pos")    
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_pos += 1
                    else: 
                        sen_actioFP.append(ls[index])
                        sen_personalIT.append(ls[index])
                        sen_purpose.append(ls[index])
                        sen_actioFP_class.append("neg")                        
                        sen_personalIT_class.append("neg")                        
                        sen_purpose_class.append("neg")                        
                        sen0_neg += 1
                        sen1_neg += 1
                        sen2_neg += 1
                                
#         if df[5][i] == "Third Party Sharing/Collection":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):                    
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_pos += 1
#                         sen2_neg += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1

#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
#                 total_data_number2.append(2)
#                 total_data_label2.append("Purpose") 
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_neg += 1
#                         sen2_pos += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1

#         if df[5][i] == "User Choice/Control":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_pos += 1
#                         sen2_neg += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1
            
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_neg += 1
#                         sen2_pos += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1

#         if df[5][i] == "Data Retention":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_pos += 1                        
#                         sen2_neg += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1
                            
                            
print("----- postive and -ve sentences")                
# print(len(sen_actioFP),len(sen_actioFP_class))
# print(len(sen_personalIT),len(sen_personalIT_class))
# print(len(sen_purpose),len(sen_purpose_class))
# print(len(sentence_list))


print(sen0_pos, sen0_neg)
print(sen1_pos, sen1_neg)
print(sen2_pos, sen2_neg)

----- postive and -ve sentences
4464 1384080
5301 1383243
5356 1383188


In [10]:
### Extraacing the test data

path = r'Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

path = r'Test'  # use your path
allFiles = glob.glob(path + "/*.csv")
print("len of all files", len(allFiles))


# all the followint positives are to be counted if the sentences are in that particular class. 
# A sen_neg to be counted in any other circumstances
test_sen0_pos = 0 # action_first_party no. of sentence positives
test_sen0_neg = 0 # action_first_party no. of sentence negatives
test_sen_actioFP = []
test_sen_actioFP_class = []

test_sen1_pos = 0 # Personal Information Type no. of sentence positives
test_sen1_neg = 0 # action_first_party no. of sentence negatives
test_sen_personalIT = []
test_sen_personalIT_class = []

test_sen2_pos = 0 # purpose no. of sentence negatives
test_sen2_neg = 0 # purpose no. of sentence negatives
test_sen_purpose = []
test_sen_purpose_class = []

choice = 0
allFiles = allFiles[:25]


def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
                start_index.append(count)
    doc = '\n'.join(text)
    return doc


for file in allFiles:
    
    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    number_of_segments = len(df) + 1

    file = file.split('Test/', 1)[1]

    ### beautifulsuop on html doc
    file_html = 'Plain_Html/'+file
    file_html = file_html.split('csv', 1)[0]
    file_html = file_html+'html'
    ls = []
    indeces = []
    data = html_to_text(file_html)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
        count = count+len(l)

    
    
    for i in range(number_of_segments-1):
        choice = 0 #whether the sentence is from one of the following categories
        if df[5][i] == "First Party Collection/Use":
            choice = 1
            parse_json = json.loads(str(df[6][i]))

            if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
                str_index = parse_json["Action First-Party"]["startIndexInSegment"]
                end_index = parse_json["Action First-Party"]["endIndexInSegment"]
                for index in range(len(ls)):
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        test_sen0_pos += 1
                        test_sen1_neg += 1
                        test_sen2_neg += 1                        
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("pos")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
                    else: 
                        test_sen0_neg += 1
                        test_sen1_neg += 1
                        test_sen2_neg += 1                                                
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
                
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                for index in range(len(ls)):
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        test_sen1_pos += 1
                        test_sen0_neg += 1
                        test_sen2_neg += 1
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("pos")                        
                        test_sen_purpose_class.append("neg")
                    else: 
                        test_sen0_neg += 1
                        test_sen1_neg += 1
                        test_sen2_neg += 1                                                
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
                
            if parse_json["Purpose"]["endIndexInSegment"] != -1:
                str_index = parse_json["Purpose"]["startIndexInSegment"]
                end_index = parse_json["Purpose"]["endIndexInSegment"]
                for index in range(len(ls)):
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
                        test_sen0_neg += 1
                        test_sen1_neg += 1                        
                        test_sen2_pos += 1
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("pos")
                    else: 
                        test_sen0_neg += 1
                        test_sen1_neg += 1
                        test_sen2_neg += 1                                                
                        test_sen_actioFP.append(ls[index])
                        test_sen_personalIT.append(ls[index])
                        test_sen_purpose.append(ls[index])
                        test_sen_actioFP_class.append("neg")                        
                        test_sen_personalIT_class.append("neg")                        
                        test_sen_purpose_class.append("neg")
#         if df[5][i] == "Third Party Sharing/Collection":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sen0_neg += 1
#                         test_sen1_pos += 1                        
#                         test_sen2_neg += 1
#                     else: 
#                         test_sen0_neg += 1
#                         test_sen1_neg += 1                        
#                         test_sen2_neg += 1

#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
#                 total_data_number2.append(2)
#                 total_data_label2.append("Purpose") 
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sen0_neg += 1
#                         test_sen1_neg += 1                        
#                         test_sen2_pos += 1
#                     else: 
#                         test_sen0_neg += 1
#                         test_sen1_neg += 1                        
#                         test_sen2_neg += 1

#         if df[5][i] == "User Choice/Control":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sen0_neg += 1
#                         test_sen1_pos += 1                        
#                         test_sen2_neg += 1
#                     else: 
#                         test_sen0_neg += 1
#                         test_sen1_neg += 1                        
#                         test_sen2_neg += 1
            
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sen0_neg += 1
#                         test_sen1_neg += 1                        
#                         test_sen2_pos += 1
#                     else: 
#                         test_sen0_neg += 1
#                         test_sen1_neg += 1                        
#                         test_sen2_neg += 1

#         if df[5][i] == "Data Retention":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         test_sen0_neg += 1
#                         test_sen1_pos += 1                        
#                         test_sen2_neg += 1
#                     else: 
#                         test_sen0_neg += 1
#                         test_sen1_neg += 1                        
#                         test_sen2_neg += 1



print("----- postive and -ve sentences count")                
# print(len(test_sen_actioFP),len(test_sen_actioFP_class))
# print(len(test_sen_personalIT),len(test_sen_personalIT_class))
# print(len(test_sen_purpose),len(test_sen_purpose_class))


print(test_sen0_pos,test_sen0_neg)
print(test_sen1_pos,test_sen1_neg)
print(test_sen2_pos,test_sen2_neg)



len of all files 25
----- postive and -ve sentences count
1791 673428
2286 672933
2175 673044


In [51]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

### randomly choosing -ve sentences from the huge list
### We shall try to maximizze the prediction

### separating the +ve and -ve sentences
classes = [1]*4464
classes[4464:8928] = [0]*4464
final_action_FP = sen_actioFP[:4464]
action_FP_neg = sen_actioFP[4465:]

final_list = []
max_result_list = []

final_list = np.random.choice(action_FP_neg, 4464)

ls = final_action_FP
ls.extend(final_list)

# performing the training here
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(ls, classes)

### performing the testing here and we will iterate the process and save the best results
test_set = test_sen_actioFP[:100]+test_sen_actioFP[1791:1891]
test_classes = [1]*100 + [0]*100
predicted = text_clf.predict(test_set)
print(np.mean(predicted == test_classes))

print(metrics.classification_report(test_classes, predicted))

metrics.confusion_matrix(test_classes, predicted)

0.5
             precision    recall  f1-score   support

          0       0.50      0.89      0.64       100
          1       0.50      0.11      0.18       100

avg / total       0.50      0.50      0.41       200



array([[89, 11],
       [89, 11]])

In [35]:
final_list = []
max_result_list = []

final_list1 = np.random.choice(action_FP_neg, 4464)

ls1 = final_action_FP
ls1.extend(final_list1)


# performing the training here
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(ls1, classes)

### performing the testing here and we will iterate the process and save the best results

predicted = text_clf.predict(test_set)
print(np.mean(predicted == test_classes))

print(metrics.classification_report(test_classes, predicted))

metrics.confusion_matrix(test_classes, predicted)

0.5
             precision    recall  f1-score   support

          0       0.50      0.88      0.64       100
          1       0.50      0.12      0.19       100

avg / total       0.50      0.50      0.42       200



array([[88, 12],
       [88, 12]])

In [36]:
classes = [1]*4464
classes[4464:8928] = [0]*4464
final_action_FP = sen_actioFP[:4464]
action_FP_neg = sen_actioFP[4465:]

final_list = []
max_result_list = []

final_list2 = np.random.choice(action_FP_neg, 4464)

ls2 = final_action_FP
ls2.extend(final_list2)


# performing the training here
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(ls2, classes)

### performing the testing here and we will iterate the process and save the best results

predicted = text_clf.predict(test_set)
print(np.mean(predicted == test_classes))

print(metrics.classification_report(test_classes, predicted))

metrics.confusion_matrix(test_classes, predicted)

0.51
             precision    recall  f1-score   support

          0       0.51      0.91      0.65       100
          1       0.55      0.11      0.18       100

avg / total       0.53      0.51      0.42       200



array([[91,  9],
       [89, 11]])

In [70]:
final_list1 = []
for i in range(10):
    print(i)
    final_list1.append(np.random.choice(action_FP_neg, 4464).tolist())


0
1
2
3
4
5
6
7
8
9


In [69]:
final_list5 = []
final_list5.append(np.random.choice(action_FP_neg, 4464).tolist())
print(len(final_list5[0]))

4464


In [78]:
ls = []
final_action_FP = sen_actioFP[:4464]

for i in range(10):
    ls = []
    ls = final_action_FP
    print(len(final_action_FP), len(ls))
    print("len is 1", len(ls), len(final_action_FP))
    ls.extend(final_list1[i])
    print("len is 2", len(ls))
    # performing the training here
    text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
    text_clf = text_clf.fit(ls, classes)

    ### performing the testing here and we will iterate the process and save the best results

    predicted = text_clf.predict(test_set)
    
    print(np.mean(predicted == test_classes))
    print(metrics.classification_report(test_classes, predicted))
    
    metrics.confusion_matrix(test_classes, predicted)
    ls[4464:8928] = []
    print("-----")

4464 4464
len is 1 4464 4464
len is 2 8928
0.495
             precision    recall  f1-score   support

          0       0.50      0.88      0.64       100
          1       0.48      0.11      0.18       100

avg / total       0.49      0.49      0.41       200

-----
4464 4464
len is 1 4464 4464
len is 2 8928
0.5
             precision    recall  f1-score   support

          0       0.50      0.86      0.63       100
          1       0.50      0.14      0.22       100

avg / total       0.50      0.50      0.43       200

-----
4464 4464
len is 1 4464 4464
len is 2 8928
0.5
             precision    recall  f1-score   support

          0       0.50      0.91      0.65       100
          1       0.50      0.09      0.15       100

avg / total       0.50      0.50      0.40       200

-----
4464 4464
len is 1 4464 4464
len is 2 8928
0.5
             precision    recall  f1-score   support

          0       0.50      0.90      0.64       100
          1       0.50      0.10      0.

In [21]:
sample_test = ["how we use and disclose that information."]
sample_class = ["neg"]
import numpy as np
predicted = text_clf.predict(sample_test)
np.mean(predicted == sample_class)

1.0

In [89]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

### randomly choosing -ve sentences from the huge list
### We shall try to maximizze the prediction

### separating the +ve and -ve sentences
classes = [1]*5301
classes[5301:10602] = [0]*5301
print(len(classes))
final_personalIT = sen_personalIT[:5301]
personalIT_neg = sen_personalIT[5302:]

final_list = []
max_result_list = []

final_list = np.random.choice(personalIT_neg, 5301)

ls_personal = final_personalIT
ls_personal.extend(final_list)

# performing the training here
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(ls_personal, classes)

### performing the testing here and we will iterate the process and save the best results
test_set = test_sen_personalIT[:100]+test_sen_personalIT[2286:2386]
test_classes = [1]*100 + [0]*100
predicted = text_clf.predict(test_set)
print(np.mean(predicted == test_classes))

print(metrics.classification_report(test_classes, predicted))

metrics.confusion_matrix(test_classes, predicted)

10602
0.49
             precision    recall  f1-score   support

          0       0.49      0.85      0.62       100
          1       0.46      0.13      0.20       100

avg / total       0.48      0.49      0.41       200



array([[85, 15],
       [87, 13]])

In [83]:
final_list2 = []
for i in range(10):
    print(i)
    final_list2.append(np.random.choice(personalIT_neg, 5301).tolist())

0
1
2
3
4
5
6
7
8
9


In [84]:
for i in range(10):
    print(len(final_list2[i]))

5301
5301
5301
5301
5301
5301
5301
5301
5301
5301


In [90]:
ls = []
final_personalIT = sen_personalIT[:5301]

for i in range(10):
    ls2 = []
    ls2 = final_personalIT
    print(len(final_personalIT), len(ls2))
    print("len is 1", len(ls2), len(final_personalIT))
    ls2.extend(final_list2[i])
    print("len is 2", len(ls2))
    # performing the training here
    text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
    text_clf = text_clf.fit(ls2, classes)

    ### performing the testing here and we will iterate the process and save the best results

    predicted = text_clf.predict(test_set)
    
    print(np.mean(predicted == test_classes))
    print(metrics.classification_report(test_classes, predicted))
    
    metrics.confusion_matrix(test_classes, predicted)
    ls2[5301:10602] = []
    print("-----")

5301 5301
len is 1 5301 5301
len is 2 10602
0.5
             precision    recall  f1-score   support

          0       0.50      0.01      0.02       100
          1       0.50      0.99      0.66       100

avg / total       0.50      0.50      0.34       200

-----
5301 5301
len is 1 5301 5301
len is 2 10602
0.5
             precision    recall  f1-score   support

          0       0.50      0.01      0.02       100
          1       0.50      0.99      0.66       100

avg / total       0.50      0.50      0.34       200

-----
5301 5301
len is 1 5301 5301
len is 2 10602
0.5
             precision    recall  f1-score   support

          0       0.50      0.01      0.02       100
          1       0.50      0.99      0.66       100

avg / total       0.50      0.50      0.34       200

-----
5301 5301
len is 1 5301 5301
len is 2 10602
0.5
             precision    recall  f1-score   support

          0       0.50      0.01      0.02       100
          1       0.50      0.99      

In [91]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

### randomly choosing -ve sentences from the huge list
### We shall try to maximizze the prediction

### separating the +ve and -ve sentences
classes = [1]*5356
classes[5356:10712] = [0]*5356
print(len(classes))
final_purpose = sen_purpose[:5356]
purpose_neg = sen_purpose[5357:]

final_list3 = []

final_list3 = np.random.choice(purpose_neg, 5356)

ls_personal = final_purpose
ls_personal.extend(final_list3)

# performing the training here
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(ls_personal, classes)

### performing the testing here and we will iterate the process and save the best results
test_set = test_sen_purpose[:100]+test_sen_purpose[2175:2275]
test_classes = [1]*100 + [0]*100
predicted = text_clf.predict(test_set)
print(np.mean(predicted == test_classes))

print(metrics.classification_report(test_classes, predicted))

metrics.confusion_matrix(test_classes, predicted)

10712
0.495
             precision    recall  f1-score   support

          0       0.50      0.86      0.63       100
          1       0.48      0.13      0.20       100

avg / total       0.49      0.49      0.42       200



array([[86, 14],
       [87, 13]])